## Initialize OpenAI Client
Set up the OpenAI client with Nebius AI's API endpoint and your API key

In [1]:
import os
from openai import OpenAI
import time

client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

## Upload Training and Validation Datasets
Nebius AI requires datasets in `.jsonl` format. Upload your training and validation datasets. In my case, I have prepared only one jsonl file, so that I will use a short script to split that file into two new files for training and validation, with the ratio of 9:1.

In [ ]:
import json
import random

input_file = r"data/dataset.jsonl"
train_file = r"data/train_dataset.jsonl"
valid_file = r"data/valid_dataset.jsonl"

with open(input_file, "r") as f:
    lines = f.readlines()

random.shuffle(lines)

split_index = int(0.9 * len(lines))
train_lines = lines[:split_index]
valid_lines = lines[split_index:]

with open(train_file, "w") as f:
    f.writelines(train_lines)

with open(valid_file, "w") as f:
    f.writelines(valid_lines)

print(f"Training dataset saved to {train_file}")
print(f"Validation dataset saved to {valid_file}")

Training dataset saved to data/train_dataset.jsonl
Validation dataset saved to data/valid_dataset.jsonl


In [17]:
# Upload a training dataset
training_dataset = client.files.create(
    file=open(train_file, "rb"), # Specify the dataset name
    purpose="fine-tune"
)

# Upload a validation dataset
validation_dataset = client.files.create(
    file=open(valid_file, "rb"), # Specify the dataset name
    purpose="fine-tune"
)

## Create Fine-Tuning Job
Specify the model and fine-tuning parameters, including number of epochs and LoRA adaptation.

If you want to add more Parameters, Check your [Nebius Docs](https://docs.nebius.com/studio/fine-tuning/how-to-fine-tune#instructions) for more information.

In [18]:
wandb_api_key = os.environ.get("WANDB_API_KEY")

In [19]:
job_request = {
    "model": "meta-llama/Llama-3.1-8B-Instruct", # Choose the model
    "training_file": training_dataset.id,
    "validation_file": validation_dataset.id,
    "hyperparameters": {
        "n_epochs": 3, # Number of epochs for training
        "batch_size": 4,
        "learning_rate": 0.0003,
        "lora": True,  # Enable LoRA for fine-tuning efficiency,
        "lora_r": 16,
        "lora_alpha": 32,
        "lora_dropout": 0.1
    },
    "seed": 42,
    "integrations": [
        {
         "type": "wandb",
         "wandb": {
            "api_key": wandb_api_key,
            "project": "llama-ft-project"
         }
      }
    ],
}

# Create and run the fine-tuning job
job = client.fine_tuning.jobs.create(**job_request)

## Monitor Fine-Tuning Status
Check the job status periodically to determine whether it is running, completed, or failed.

In [ ]:
# Create and run the fine-tuning job
job = client.fine_tuning.jobs.create(**job_request)

# Make sure that the job has been finished or cancelled
active_statuses = ["validating_files", "queued", "running"]
while job.status in active_statuses:
    time.sleep(15)
    job = client.fine_tuning.jobs.retrieve(job.id)
    print("current status is", job.status)

print("Job ID:", job.id)

## Retrieve Fine-Tuned Model Checkpoints
If the fine-tuning job succeeds, retrieve the trained model checkpoints.

In [ ]:
if job.status == "succeeded":
    # Check the job events
    events = client.fine_tuning.jobs.list_events(job.id)
    print(events)

    for checkpoint in client.fine_tuning.jobs.checkpoints.list(job.id).data:
        print("Checkpoint ID:", checkpoint.id)

        # Create a directory for every checkpoint
        os.makedirs(checkpoint.id, exist_ok=True)

        for model_file_id in checkpoint.result_files:
            # Get the name of a model file
            filename = client.files.retrieve(model_file_id).filename

            # Retrieve the contents of the file
            file_content = client.files.content(model_file_id)

            # Save the contents into a local file
            file_content.write_to_file(filename)

## Deploy the fine-tuned model

After fine-tuning, I have deployed the model on Nebius AI Studio.

## Chat with the fine-tuned model

In [ ]:
import os
from openai import OpenAI
from IPython.display import Markdown, display

client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY")
)

response = client.chat.completions.create(
    model=os.environ.get("MODEL_ID"),
    messages=[
        {"role": "system",
         "content": "You are an expert in Generative AI. In particular, you have strong understanding of four subtopics: Responsible AI, Agentic AI, Prompt Engineering and Foundation models.\nYou are given a fact-based and exam-style question about one of these subtopics. Write a response that is the best possible answer to the given question. The answer MUST have:\n- High accuracy: factually accurate, well-researched and precise.\n- Maximum comprehensiveness: long, nuanced, and detailed.\n- Diverse viewpoints, key points and perspectives: Include multiple key aspects and perspectives with the discussions on each.\n- Well-structured output: Use Markdown for clear formatting and better readability.\n- No hallucination: Stick to verifiable facts. Do not make up any information."
        },
        {"role": "user",
         "content": "What is Agentic AI?"
        }
    ],
    top_p=0.5,
    max_tokens=1024,
)

ft_model_generation = response.choices[0].message.content

display(Markdown(ft_model_generation))

## Understanding Agentic AI

#### Key Characteristics of Agentic AI

- **Definition**: Agentic AI refers to a type of AI system that possesses characteristics of autonomy, self-awareness, and social intelligence, enabling it to interact with humans and other agents in a more human-like and adaptive manner.

- **Core Components**:
  - Perception and understanding
  - Self-awareness and meta-cognition
  - Adaptive behavior and learning
  - Social interaction and communication

#### Key Characteristics of Agentic AI Systems

- **Autonomous Decision-Making**:
  - Ability to make decisions without human intervention
  - Self-directed learning and adaptation
  - Real-time response generation

- **Social Intelligence**:
  - Understanding human emotions and behavior
  - Developing social relationships
  - Cultural adaptation capabilities

#### Implementation Approaches

- **Architecture**:
  - Modular design with multiple components
  - Neural network structure for learning
  - Hybrid approaches combining symbolic and connectionist AI

- **Training Methods**:
  - Reinforcement learning for behavior
  - Imitation learning from human demonstrations
  - Multi-task learning for diverse capabilities

#### Applications and Examples

- **Virtual Assistants**: Siri, Alexa, Google Assistant
- **Game AI**: AI opponents in strategy games
- **Customer Service AI**: Chatbots with human-like conversation

#### Challenges and Considerations

- **Ethical Implications**:
  - Transparency and accountability
  - Bias and fairness
  - Privacy and consent

- **Technical Challenges**:
  - Scalability and reliability
  - Communication efficiency
  - System safety and security

## Conclusion
Agentic AI represents a significant advancement in AI capabilities, combining human-like intelligence with autonomous capabilities. Its development requires careful consideration of multiple technical, ethical, and practical aspects.